In [ ]:
!nvidia-smi

Sun Jun 20 20:46:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive/")

Mounted at /content/MyDrive/


In [ ]:
import  os
import torch
from torchvision.io import read_image
from torchvision.io import read_image
from torchvision.transforms import functional as F
import time

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


#Dataset, loader and augmentation

In [ ]:
data_dir = "/content/MyDrive/MyDrive/PFE_Deepfakes/datasets/DFDC/splited/train_validation_test"

In [ ]:
class CustomImageDatasetAndAugmenter():
    def __init__(self, label, img_dir, p_hflip=0, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1) ):
        self.img_label = label
        self.img_dir = img_dir
        self.p_hflip = p_hflip
        self.mean = mean
        self.std = std
        self.image_shape = image_shape
        self.brightness = brightness
        self.contrast = contrast
        self.saturation = saturation
        self.hue = hue
        

    def __len__(self):
        return len(os.listdir(self.img_dir))

    def __getitem__(self, idx):
        list_images = os.listdir(self.img_dir)
        file_name = list_images[idx]
        img_path_base = os.path.join(self.img_dir, file_name)
        if len(file) ==28 or len(file) ==17 :
          next_frame_number = str(int(file_name[-6:-4]) +1)
        if len(file) ==27 or len(file) ==16 : 
          next_frame_number = str(int(file_name[-5]) +1)
        next_frame_name = file_name[0:22] + next_frame_number + ".jpg"
        next_frame_path = os.path.join(self.img_dir,next_frame_name)
        if os.path.isfile(next_frame_path) :
          base_image = read_image(img_path_base)
          next_image = read_image(next_frame_path)

          #H_flip
          if torch.rand(1) < self.p_hflip :
            base_image = F.hflip(base_image)
            next_image = F.hflip(next_image)
          
          #Colors transforms
          brightness_factor = None if self.brightness is None else float(torch.empty(1).uniform_(self.brightness[0], self.brightness[1]))
          contrast_factor = None if self.contrast is None else float(torch.empty(1).uniform_(self.contrast[0], self.contrast[1]))
          saturation_factor = None if self.saturation is None else float(torch.empty(1).uniform_(self.saturation[0], self.saturation[1]))
          hue_factor = None if self.hue is None else float(torch.FloatTensor(1).uniform_(self.hue[0], self.hue[1]))


          fn_idx = torch.randperm(4) #to do the different transformation in a random order

          for fn_id in fn_idx:
            if fn_id == 0 and brightness_factor is not None:
                base_image = F.adjust_brightness(base_image, brightness_factor)
                next_image = F.adjust_brightness(next_image, brightness_factor)
            elif fn_id == 1 and contrast_factor is not None:
                base_image = F.adjust_contrast(base_image, contrast_factor)
                next_image = F.adjust_contrast(next_image, contrast_factor)
            elif fn_id == 2 and saturation_factor is not None:
                base_image = F.adjust_saturation(base_image, saturation_factor)
                next_image = F.adjust_saturation(next_image, saturation_factor)
            elif fn_id == 3 and hue_factor is not None:
                base_image = F.adjust_hue(base_image, hue_factor)
                next_image = F.adjust_hue(next_image, hue_factor)

          #Stack the images
          stacked_tensor = torch.stack((base_image, next_image), dim=3)

          #Normalization
          mean = self.mean
          R_c = torch.ones(self.image_shape)*mean[0]
          G_c = torch.ones(self.image_shape)*mean[1]
          B_c = torch.ones(self.image_shape)*mean[2]
          RGB_3c = torch.stack((R_c, G_c, B_c), dim=2) 
          mean_3c_RGB = RGB_3c.permute(2, 0, 1)
          stacked_mean = torch.stack((mean_3c_RGB, mean_3c_RGB), dim=3)
          std = self.std 
          std_R_c = torch.ones(self.image_shape)*mean[0]
          std_G_c = torch.ones(self.image_shape)*mean[1]
          std_B_c = torch.ones(self.image_shape)*mean[2]
          std_RGB_3c = torch.stack((R_c, G_c, B_c), dim=2) 
          std_3c_RGB = std_RGB_3c.permute(2, 0, 1)
          stacked_std = torch.stack((std_3c_RGB, std_3c_RGB), dim=3)

          normalized_tensor = torch.div(torch.sub(stacked_tensor/255, stacked_mean), stacked_std)

          label = self.img_label

          return normalized_tensor, label

####train

In [ ]:
train_fake_dataset = CustomImageDatasetAndAugmenter(0, os.path.join(data_dir, "train","fake"), p_hflip=0.5, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))

In [ ]:
train_real_dataset = CustomImageDatasetAndAugmenter(1, os.path.join(data_dir, "train","real"), p_hflip=0.5, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))

In [ ]:
try : 
  train_dataset = torch.utils.data.ConcatDataset((train_fake_dataset, train_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  train_dataset = torch.utils.data.ConcatDataset((train_fake_dataset, train_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  train_dataset = torch.utils.data.ConcatDataset((train_fake_dataset, train_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  train_dataset = torch.utils.data.ConcatDataset((train_fake_dataset, train_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  train_dataset = torch.utils.data.ConcatDataset((train_fake_dataset, train_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


####validation

In [ ]:
validation_fake_dataset = CustomImageDatasetAndAugmenter(0, os.path.join(data_dir, "validation","fake"), p_hflip=0, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=None, contrast=None, saturation=None, hue=None)

In [ ]:
validation_real_dataset = CustomImageDatasetAndAugmenter(1, os.path.join(data_dir, "validation","real"), p_hflip=0, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=None, contrast=None, saturation=None, hue=None)

In [ ]:
try : 
  validation_dataset = torch.utils.data.ConcatDataset((validation_fake_dataset, validation_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  validation_dataset = torch.utils.data.ConcatDataset((validation_fake_dataset, validation_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  validation_dataset = torch.utils.data.ConcatDataset((validation_fake_dataset, validation_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


In [ ]:
try : 
  validation_dataset = torch.utils.data.ConcatDataset((validation_fake_dataset, validation_real_dataset))
  print("Done !")
except : 
  print("Not yet...")

Done !


####test

In [ ]:
test_fake_dataset = CustomImageDatasetAndAugmenter(0, os.path.join(data_dir, "test","fake"), p_hflip=0, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=None, contrast=None, saturation=None, hue=None)

In [ ]:
test_real_dataset = CustomImageDatasetAndAugmenter(1, os.path.join(data_dir, "test","real"), p_hflip=0, mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), image_shape = (224,224), brightness=None, contrast=None, saturation=None, hue=None)

In [ ]:
test_dataset = torch.utils.data.ConcatDataset((test_fake_dataset, test_real_dataset))

####Dataloader

In [ ]:
image_datasets = {
        "train": train_dataset,
        "validation": validation_dataset,
        "test": test_dataset
    }

In [ ]:
def load_gpu(image_datasets, batch_size, data_dir):
    
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size,
                                                 shuffle=True, num_workers=0, pin_memory=True)
                   for x in ['train', 'validation', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation', 'test']}
    
    return dataloaders, dataset_sizes

In [ ]:
batch_size = 32 #https://medium.com/deep-learning-experiments/effect-of-batch-size-on-neural-net-training-c5ae8516e57 / paper : https://arxiv.org/abs/1609.04836
batch_size_512 = 512

dataloaders, dataset_sizes = load_gpu(image_datasets, batch_size, data_dir)
dataloaders_512, dataset_sizes_512 = load_gpu(image_datasets, batch_size_512, data_dir) # works only with the high RAM option
# return batch_size, dataloaders, dataset_sizes

#Model

In [ ]:
pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
from torch import nn

In [ ]:
class EarlyFusionModel(nn.Module):
    def __init__(self):
        super(EarlyFusionModel, self).__init__()
        self.l1 = torch.nn.Conv2d(3, 3, 1)
        
    def forward(self, x):
        x = self.l1(x)
        return x



      

In [ ]:
class ConcatModels(nn.Module):
    def __init__(self, modelA, modelB):
        super(ConcatModels, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x1)
        return x2

In [ ]:
modelA = EarlyFusionModel()
modelB = EfficientNet.from_name('efficientnet-b3')

model = ConcatModels(modelA, modelB).to(device)

In [ ]:
print(model)

ConcatModels(
  (modelA): EarlyFusionModel(
    (l1): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  )
  (modelB): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40,

In [ ]:
#checkpoint = torch.load('/content/MyDrive/MyDrive/PFE_Deepfakes/df_code/EfficientNet/weigths/EfNet_dfd_epoch_0_batch_5700_2.pth')
#model.load_state_dict(checkpoint['model'])
#model.eval()
#print("Done")

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    since = time.time()
    #batch_start = 1200
    for batch, (x, y) in enumerate(dataloader):
      #if batch > batch_start +50 : 
        x = x.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(x)
        loss = loss_fn(pred, y)
        print(loss.item())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            time_elapsed = time.time() - since
            print('Those 100 batch took {:.0f}m {:.0f}s to complete'.format(time_elapsed // 60, time_elapsed % 60))
            since = time.time()

            saving_start = time.time()
            save_checkpoint_file_path = "/content/MyDrive/MyDrive/PFE_Deepfakes/df_code/EfficientNet/weigths/EfNet_dfd_epoch_{}_batch_{}_2.pth".format(epoch_start + t, batch)
            checkpoint = { 
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()}
            torch.save(checkpoint, save_checkpoint_file_path)
            saving_time = time.time() - saving_start
            print('This checkpoint {} took {:.0f}m {:.0f}s to be saved'.format(save_checkpoint_file_path, saving_time // 60, saving_time % 60))


def test_loop(dataloader, model, loss_fn):
    #size = len(dataloader.dataset) #does that count the number of elements in the dataset or the number of batches > I think the number of elements instead of the number of batches hence the bug
    #size = len(dataloader.dataset) / batch_size
    test_loss, correct = 0, 0
    i = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            print(loss.item())
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            i+=1

    test_loss /= i
    correct /= len(dataloader.dataset)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
lr = 1e-4
m= 0.9

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)    

optimizer = torch.optim.Adam((p for p in model.parameters() if p.requires_grad), lr=lr)
#optimizer.load_state_dict(checkpoint["optimizer"])

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr / 10, steps_per_epoch=len(dataloaders["train"]), epochs=10)
#scheduler.load_state_dict(checkpoint["scheduler"])

#epoch_start = checkpoint["epoch"]
#epoch_start = 0
epochs = 50
#for t in range(epochs-epoch_start):
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    epoch = t #+ epoch_start
    train_loop(dataloaders["train"], model, loss_fn, optimizer)
    save_checkpoint_file_path = "/content/MyDrive/MyDrive/PFE_Deepfakes/df_code/EfficientNet/weigths/EfNet_dfd_epoch_{}_completed_2.pth".format(epoch_start + t)
    checkpoint = { 
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()}
    torch.save(checkpoint, save_checkpoint_file_path)
    test_loop(dataloaders["test"], model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


TypeError: ignored

#Comments / to do : 

-I must readjust the dataset creation so that image 10 don't get stacked with image 2
-fake file name have a length of 27 while real file name have a len of 16. I did not take the len of the real files name into account

#Draft / test

In [ ]:
train_files = os.listdir(os.path.join(data_dir, "train", "real"))

In [ ]:
for file in train_files :
    if len(file) ==28 or len(file) ==17 : 
      print(file[-6:-4])
    if len(file) ==27 or len(file) ==16 : 
      print(file[-5])

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
7
8
0
1
2
3
4
5
6
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
0
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
0
0
1
2
3
4
5
6
7
8
9
0
1
2
0
1
2
3
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
0
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
0
1
2
3
4
5
6
7
8
9
0
1
2
3
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
0
1
0
1
2
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
0
1
2
3
4
5
6
7
8
9
0
1
0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
0
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
0
1
2
3
4
5
6
7
8
9
0
0